In [1]:
!date

Mon Sep  5 13:57:42 EDT 2022


In [2]:
pwd

'/mmfs1/data/pijarj/BC-ORG-Data/Code'

In [3]:
cd ../Data

/mmfs1/data/pijarj/BC-ORG-Data/Data


In [4]:
%%time
import pandas as pd
import os
import shutil
import numpy as np
from tqdm import tqdm
import ants
import sys
import nibabel as nib

CPU times: user 1.81 s, sys: 1.1 s, total: 2.9 s
Wall time: 35.8 s


In [5]:
#make txt file into csv
read_file = pd.read_csv ('/mmfs1/data/pijarj/NDAR_BoldAnat10/image03.txt',delimiter='\t',low_memory=False)
read_file.to_csv ('/mmfs1/data/pijarj/BC-ORG-Data/Data/image03.csv', index=0)

In [5]:
# Drops non nii or nii.gz types
df = pd.read_csv('image03.csv',low_memory=False)
df = df.iloc[1::] # First row is just another index, let's drop it
df['image_file'] = df['image_file'].astype(str) # convert entries to strings (so nan become 'nan')

df['is_nii_gz'] = [str(file).endswith('.nii.gz') for file in df['image_file'].values]
df['is_nii'] = [str(file).endswith('.nii') for file in df['image_file'].values]

# only keep the niftis 
df = df.iloc[df['is_nii_gz'].values + df['is_nii'].values]

In [1]:
args = sys.argv
script_name = sys.argv[0]
study_id = sys.argv[1]
subject_id = sys.argv[2]

In [1]:
print(f'name of this script is: {script_name}')

In [1]:
#making new folder and getting study id
print(f'study name is: {study_id}')
#os.mkdir('ds-{subject_id}')

In [1]:
def safe_mkdir(path):
    if not os.path.exists(path):
        os.mkdir('ds-{subject_id}')
    else:
        pass

In [6]:
#make directory
ndar_root = '/mmfs1/data/pijarj/NDAR_BoldAnat10/' 
bids_root = '/mmfs1/data/pijarj/'
safe_mkdir(os.path.join(bids_root,f'DS{col_id}.csv'))
for s in tqdm(range(1,nsubjects+1)):
    safe_mkdir(os.path.join(bids_root,f'ds-{col_id}',f'sub-{s:03d}'))
    safe_mkdir(os.path.join(bids_root,f'ds-{col_id}',f'sub-{s:03d}','func'))
    safe_mkdir(os.path.join(bids_root,f'ds-{col_id}',f'sub-{s:03d}','anat'))

name of this script is: /data/pijarj/anaconda3/lib/python3.8/site-packages/ipykernel_launcher.py
study name is: -f
subject number is: /mmfs1/data/pijarj/.local/share/jupyter/runtime/kernel-822f65e5-eb39-47b9-9d60-52790d2bfa7b.json


In [18]:
#making local paths column
#df = pd.read_csv(f'DS{col_id}.csv')
n = len(study_df)
#root = './image03/'
root = os.path.expanduser('~/NDAR_BoldAnat10/image03/')
local_paths = list()
for i in tqdm(range(n)):
    s3_path = study_df['image_file'].values[i]
    splits = s3_path.split('/')[4::]
    relative = '/'.join(splits)
    local_path = os.path.join(root,relative)
    assert os.path.exists(os.path.join(root,relative))
    local_paths.append(local_path)
study_df['local_paths'] = local_paths

100%|██████████| 192/192 [00:00<00:00, 961.85it/s]


In [1]:
#copying over subject
print(f'subject number is: {subject_id}')
sub = use_subjects[subject_id]
sub_df = study_df.iloc[study_df['subjectkey'].values==sub]
fmri_idx = sub_df['scan_type'].values=='fMRI'
anat_idx = sub_df['scan_type'].isin(['MR structural (T1)','MR structural (MPRAGE)']).values
epi_fn = sub_df.iloc[fmri_idx]['local_paths'].values[0]
anat_fn = sub_df.iloc[anat_idx]['local_paths'].values[0]
epi_path = os.path.join(ndar_root,epi_fn)
anat_path = os.path.join(ndar_root,anat_fn)
epi_dest = os.path.join(bids_root,f'ds-{col_id}',f'sub-{s+1:03d}','func',f'sub-{s+1:03d}_task-rest_bold.nii.gz')
anat_dest = os.path.join(bids_root,f'ds-{col_id}',f'sub-{s+1:03d}','anat',f'sub-{s+1:03d}_T1w.nii.gz')
t1 = ants.image_read(anat_path) 
bold = ants.image_read(epi_path)
t1.to_filename(anat_dest)
bold.to_filename(epi_dest)

In [12]:
def write_json(data,filepath):
    import json
    with open(filepath, 'w') as outfile:
        json.dump(data, outfile)

In [1]:
bold_json = {"RepetitionTime" : bold.spacing[-1],
         "TaskName" : 'rest'}
write_json(bold_json,epi_dest.replace('.nii.gz','.json')) 
im = nib.load(epi_dest)
header = im.header.copy()
header.set_xyzt_units(xyz='mm', t='sec')
nib.nifti1.Nifti1Image(im.get_fdata(), None, header=header).to_filename(epi_dest)
assert nib.load(epi_dest).header.get_xyzt_units()==('mm', 'sec'),'timing missing from header'

In [1]:
import json
data = {
    "Name" : study_df["collection_title"].values[0] ,
    "RepetitionTime": 2.0,
    "SliceTiming" : 2.0 ,
    "TaskName" : "taskrest" ,
    "BIDSVersion" : "20.2.0"}
json_string = json.dumps(data)
print(json_string)
with open(os.path.join(bids_root,f'ds-{col_id}','dataset_description.json'), 'w') as outfile:
    json.dump(json_string, outfile)